In [2]:
#install the necessary libraries for python and Keras
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!pip install -q keras
!pip install -q sklearn
!pip install -q keras-pandas #https://pypi.org/project/keras-pandas/

E: Package 'python-software-properties' has no installation candidate


In [3]:
#perform authorization to access Google Drive 
from google.colab import drive
#mount you GoogleDrive to "drive" folder
drive.mount('/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [0]:
!ls /drive

In [0]:
!ls /drive/'My Drive'

In [0]:
import pandas as pd
import numpy as np
#from sklearn import preprocessing

path = "/drive/My Drive/99.MyDev/Python_BTC_Predictive_Using_Deeplearning/"

df = pd.read_excel(path+'/Data/00Data_Cleansing.xlsx')

y = df.iloc[:, 35].values #numpy.ndarray
X = df.iloc[:, 1:35].values #numpy.ndarray

_input_dim = X.shape[1]
#print(X.shape[1])

In [0]:
# one hot encode outputs covert output เป็น Category
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)

#print(y[0])

y = np_utils.to_categorical(y)
num_classes = y.shape[1]

#print(num_classes)

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

print(X_train)

[[2.90400e+02 1.46850e+05 7.40700e-01 ... 1.49065e+03 7.18000e+03
  2.09960e+03]
 [6.62800e+02 7.58900e+04 7.56800e-01 ... 1.45565e+03 1.26800e+04
  2.12990e+03]
 [1.26000e+01 4.18800e+04 1.04800e+00 ... 1.28546e+03 6.91000e+03
  1.46105e+03]
 ...
 [4.52500e+02 3.99700e+04 7.37600e-01 ... 1.38241e+03 1.05300e+04
  2.06446e+03]
 [6.99000e+01 7.32000e+04 1.04440e+00 ... 1.47897e+03 3.56200e+04
  1.55689e+03]
 [1.56321e+04 1.63740e+05 7.70200e-01 ... 1.73691e+03 7.66000e+03
  2.68457e+03]]


In [86]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train)

[[-4.37013624e-01 -2.43497392e-02 -9.92226810e-01 ...  2.22907756e-01
  -4.13602838e-01  2.83968772e-01]
 [-3.14615300e-01 -1.19147118e-01 -8.59644276e-01 ...  4.47280128e-02
   5.36987583e-01  3.50707250e-01]
 [-5.28319355e-01 -1.64581995e-01  1.53837025e+00 ... -8.21683714e-01
  -4.60268186e-01 -1.12249510e+00]
 ...
 [-3.83735513e-01 -1.67133616e-01 -1.01775512e+00 ... -3.28125826e-01
   1.65393146e-01  2.06569758e-01]
 [-5.09486315e-01 -1.22740762e-01  1.50872446e+00 ...  1.63446630e-01
   4.50181381e+00 -9.11398867e-01]
 [ 4.60540947e+00 -1.78593235e-03 -7.49296081e-01 ...  1.47658043e+00
  -3.30642219e-01  1.57241787e+00]]


In [0]:
#import classes we will need เรียกใช้ library
from keras.models import Sequential #Sequential ของ layer
from keras.layers import Dense #Dense คือ ทุก node จะส่ง output ไปที่ Input ของ node ถัดไป
from keras.layers import Dropout #Dropout คือในการ trainแต่ละรอบจะสุ่มหยิบบางโหนดออก เพื่อให้ NN ไม่จำคำตอบ

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed) #กำหนด seen ให้ผลลัพธ์ใกล้เคียงกัน

In [0]:
# define baseline model
def baseline_model(): #ประกาศ Function
	# create model
	model = Sequential() #define เป็น Sequential model
	model.add(Dense(2, input_dim=_input_dim, kernel_initializer='normal', activation='relu')) # add layer ที่ 1 100 node, input layer มีขนาดเท่ากับ ตัวแปร num_pixels ใช้ activation เป็น Relu
	model.add(Dense(6,kernel_initializer='normal', activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax')) # add layser ที่ 2 ใช้ activation เป็น softmax ที่จะทำให้ pop ของการทำนาย output layer ทุก class กลายเป็น 1 จะทำให้เข้าใจง่าย
	# Compile model;  https://keras.io/losses/ for more details 
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #ระบุ loss ของ output เป็น categorical_crossentropy และดู accuracy
	return model

In [551]:
# build the model
model = baseline_model()
print(model.summary()) #print โครงสร้างของ Network layer 1 มี 100 node param คำนวนจาก 100*784 = 78400 แสดง 78500 , 100 มาจาก bios แต่ละ node จะมี bios

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_243 (Dense)            (None, 2)                 70        
_________________________________________________________________
dense_244 (Dense)            (None, 6)                 18        
_________________________________________________________________
dense_245 (Dense)            (None, 2)                 14        
Total params: 102
Trainable params: 102
Non-trainable params: 0
_________________________________________________________________
None


In [557]:
from keras.callbacks import EarlyStopping #คือ จะให้ เทสเท่าไร ถ้าไม่ดีขึ้นให้หยุด แต่โดยทั่วไปไม่ใช้ EarlyStoping แต่จะใช้วิธี run ทุกรอบแล้วเก็บรอบที่ดีมาใช้งาน
early_stopping_monitor = EarlyStopping(patience=5) #stop training if val_acc is not improved in next 3 epochs
# Fit the model; คือการปรับ เวท
# https://keras.io/models/sequential/ for more details
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, verbose=2,callbacks=[early_stopping_monitor]) #จะมีการปรับ weigth , validation_data เป็น data ที่เอาไว้ test ว่า model เป็นไง ,epochs ทำกี่รอบ , batch_size คือ รอบ wigth ทุกๆ input 64 ตัว, verbose เป็นการ show ค่า มี 0 1 2
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 1263 samples, validate on 316 samples
Epoch 1/50
 - 0s - loss: 0.6698 - acc: 0.5748 - val_loss: 0.7003 - val_acc: 0.5696
Epoch 2/50
 - 0s - loss: 0.6696 - acc: 0.5788 - val_loss: 0.6997 - val_acc: 0.5665
Epoch 3/50
 - 0s - loss: 0.6694 - acc: 0.5756 - val_loss: 0.7006 - val_acc: 0.5633
Epoch 4/50
 - 0s - loss: 0.6690 - acc: 0.5788 - val_loss: 0.7006 - val_acc: 0.5601
Epoch 5/50
 - 0s - loss: 0.6697 - acc: 0.5756 - val_loss: 0.7000 - val_acc: 0.5665
Epoch 6/50
 - 0s - loss: 0.6686 - acc: 0.5788 - val_loss: 0.7012 - val_acc: 0.5633
Epoch 7/50
 - 0s - loss: 0.6685 - acc: 0.5796 - val_loss: 0.7023 - val_acc: 0.5601
Baseline Error: 43.99%
